<a href="https://colab.research.google.com/github/GMinnu/AI-Based-Cyber-Security-Threats-Prediction/blob/main/SQL_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sqlite3

# Create a new database in memory or file
conn = sqlite3.connect(':memory:')  # Use filename e.g. 'school.db' for file-based

cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE Students (
  student_id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  email TEXT,
  phone_number TEXT,
  enrollment_date DATE DEFAULT (DATE('now'))
);
''')

cursor.execute('''
CREATE TABLE Courses (
  course_id INTEGER PRIMARY KEY AUTOINCREMENT,
  course_name TEXT NOT NULL,
  instructor TEXT NOT NULL
);
''')

cursor.execute('''
CREATE TABLE Enrollment (
  enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
  student_id INTEGER,
  course_id INTEGER,
  grade TEXT,
  FOREIGN KEY(student_id) REFERENCES Students(student_id),
  FOREIGN KEY(course_id) REFERENCES Courses(course_id)
);
''')

# Insert sample data
cursor.executemany('''
INSERT INTO Students (name, email, phone_number) VALUES (?, ?, ?)
''', [
    ('Alice', 'alice@gmail.com', '1234567890'),
    ('Bob', 'bob@gmail.com', '2345678901'),
    ('Charlie', 'charlie@gmail.com', '3456789012')
])

cursor.executemany('''
INSERT INTO Courses (course_name, instructor) VALUES (?, ?)
''', [
    ('Math', 'Prof. Euler'),
    ('Physics', 'Prof. Newton'),
    ('Chemistry', 'Prof. Curie')
])

cursor.executemany('''
INSERT INTO Enrollment (student_id, course_id, grade) VALUES (?, ?, ?)
''', [
    (1, 1, 'A'),
    (1, 2, 'B'),
    (2, 1, 'C'),
    (3, 3, 'A')
])

conn.commit()

# Define a helper function to run and print query results
def run_query(sql, params=()):
    cursor.execute(sql, params)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    print('\n')

In [7]:
# 1. Select all students
print("All Students:")
run_query("SELECT * FROM Students;")

All Students:
(1, 'Alice', 'alice@gmail.com', '1234567890', '2025-10-28')
(2, 'Bob', 'bob@gmail.com', '2345678901', '2025-10-28')
(3, 'Charlie', 'charlie@gmail.com', '3456789012', '2025-10-28')




In [8]:
# 2. Get student names and emails
print("Student names and emails:")
run_query("SELECT name, email FROM Students;")


Student names and emails:
('Alice', 'alice@gmail.com')
('Bob', 'bob@gmail.com')
('Charlie', 'charlie@gmail.com')




In [9]:
# 3. Add a new student
print("Adding David:")
cursor.execute("INSERT INTO Students (name, email, phone_number) VALUES (?, ?, ?)",
               ('David', 'david@gmail.com', '4567890123'))
conn.commit()
run_query("SELECT * FROM Students;")


Adding David:
(1, 'Alice', 'alice@gmail.com', '1234567890', '2025-10-28')
(2, 'Bob', 'bob@gmail.com', '2345678901', '2025-10-28')
(3, 'Charlie', 'charlie@gmail.com', '3456789012', '2025-10-28')
(4, 'David', 'david@gmail.com', '4567890123', '2025-10-28')




In [10]:
# 4. Update a student’s phone number
print("Updating Alice's phone number:")
cursor.execute("UPDATE Students SET phone_number = ? WHERE name = ?", ('9876543210', 'Alice'))
conn.commit()
run_query("SELECT * FROM Students WHERE name = 'Alice';")

Updating Alice's phone number:
(1, 'Alice', 'alice@gmail.com', '9876543210', '2025-10-28')




In [11]:
# 5. Delete a student record
print("Deleting Charlie:")
cursor.execute("DELETE FROM Students WHERE name = 'Charlie'")
conn.commit()
run_query("SELECT * FROM Students;")

Deleting Charlie:
(1, 'Alice', 'alice@gmail.com', '9876543210', '2025-10-28')
(2, 'Bob', 'bob@gmail.com', '2345678901', '2025-10-28')
(4, 'David', 'david@gmail.com', '4567890123', '2025-10-28')




In [12]:
# 6. List all courses offered
print("All Courses:")
run_query("SELECT * FROM Courses;")

All Courses:
(1, 'Math', 'Prof. Euler')
(2, 'Physics', 'Prof. Newton')
(3, 'Chemistry', 'Prof. Curie')




In [13]:
# 7. Find students enrolled in Math
print("Students enrolled in Math:")
run_query('''
SELECT Students.name
FROM Students
JOIN Enrollment ON Students.student_id = Enrollment.student_id
JOIN Courses ON Courses.course_id = Enrollment.course_id
WHERE Courses.course_name = 'Math';
''')

Students enrolled in Math:
('Alice',)
('Bob',)




In [14]:
# 8. Get the grades of all students for Physics
print("Grades for Physics course:")
run_query('''
SELECT Students.name, Enrollment.grade
FROM Students
JOIN Enrollment ON Students.student_id = Enrollment.student_id
JOIN Courses ON Courses.course_id = Enrollment.course_id
WHERE Courses.course_name = 'Physics';
''')

Grades for Physics course:
('Alice', 'B')




In [15]:
# 9. Count of students in each course
print("Student count per course:")
run_query('''
SELECT Courses.course_name, COUNT(Enrollment.student_id) AS student_count
FROM Courses
LEFT JOIN Enrollment ON Courses.course_id = Enrollment.course_id
GROUP BY Courses.course_name;
''')

Student count per course:
('Chemistry', 1)
('Math', 2)
('Physics', 1)




In [16]:
# 10. Find students with grade 'A'
print("Students with grade A:")
run_query('''
SELECT Students.name, Courses.course_name
FROM Students
JOIN Enrollment ON Students.student_id = Enrollment.student_id
JOIN Courses ON Courses.course_id = Enrollment.course_id
WHERE Enrollment.grade = 'A';
''')

Students with grade A:
('Alice', 'Math')




In [17]:
# 11. Advanced: List students not enrolled in any course
print("Students not enrolled in any course:")
run_query('''
SELECT name FROM Students
WHERE student_id NOT IN (SELECT student_id FROM Enrollment);
''')

Students not enrolled in any course:
('David',)




In [18]:
# 12. Get most recent enrollment date per student
print("Most recent enrollment date per student:")
run_query('''
SELECT name, MAX(enrollment_date) AS latest_enrollment
FROM Students
GROUP BY name;
''')

Most recent enrollment date per student:
('Alice', '2025-10-28')
('Bob', '2025-10-28')
('David', '2025-10-28')




In [19]:
# Close connection (optional)
conn.close()